# Wprowadzenie do obliczeń symbolicznych i numerycznych w Pythonie
Będziemy potrzebowali bibliotek `numpy` oraz `sympy`. Jeżeli ich nie masz, to zainstaluj je za pomocą
menedżera modułów (pip lub Conda w zależności czego używasz).

Pierwsza to biblioteka do obliczeń numerycznych, druga to biblioteka do obliczeń symbolicznych. Obydwie przydadzą
się nam do rozwiązywania równań różniczkowych metodą Ritza oraz Galerkina.

Na wstępie przyjrzymy się modułowi do obliczeń symbolicznych.

In [23]:
from sympy import *
# definijemy symbole
x, y = symbols("x y")

In [93]:
z = (x * y / (x + y)) ** 2
print(z)
print('Pochodna z z po x to:')
print(diff(z, x))
print('Całka z po y to:')
print(integrate(z, y))
print('Całka podwójna oznaczona y od 1 do 2, x od 0 do 1:')
result = integrate(z, (y, 1, 2), (x, 0, 1))
print(result)
print('Wynik numeryczny:')
print(N(result))

x**2*y**2/(x + y)**2
Pochodna z z po x to:
-2*x**2*y**2/(x + y)**3 + 2*x*y**2/(x + y)**2
Całka z po y to:
x**2*(-x**2/(x + y) - 2*x*log(x + y) + y)
Całka podwójna oznaczona y od 1 do 2, x od 0 do 1:
-17*log(3)/2 + 13/4 + 9*log(2)
Wynik numeryczny:
0.150120171360575


## Zadanie 1.
Oblicz pochodną cząstkową funkcji $f(x,y)=\frac{ln(x+1) \cdot y}{\left(x+y\right)^2}$ po zmiennej x w punkcie $(x,y)=(4,4)$.

Wskazówka: podstawienie - https://docs.sympy.org/latest/tutorial/basic_operations.html

## Rozwiązywanie układów równań liniowych
Układ rówań liniowych można przedstawić w postaci równania macierzowego:

$Ax=b$

Zapewne znane są wam metody rozwiązywania układów równań. Poznamy tutaj tylko sposób jak je wywołać w bibliotece sympy.

In [41]:
import numpy as np
# rozwiązanie symboliczne za pomocą sympy
system = Matrix(([1, 2, 3], [3, 3, 0]))
print(linsolve(system, (x, y)))
# rozwiązanie numeryczne za pomocą numpy
A = np.array([[1,2], [3,3]])
b = np.array([3,0])
print(np.linalg.solve(A, b))

{(-3, 3)}
[-3.  3.]


## Numeryczne rozwiązywanie równań różniczkowych
Mamy następujący problem:

$Au=f$ w $D \subset R^n$

$u|_{\partial D}=0$ - jednorodne warunki brzegowe

W wypadku omawianych dzisiaj metod (Ritza oraz Galerkina) będziemy potrzebowali funkcji bazowych (zwanych też funkcjami kształtu, funkcjami podstawowymi).
Następnie będziemy szukali takiej ich kombinacji liniowej, aby jak najlepiej przybliżyć rozwiązanie, czyli poszukujemy takich $\left\lbrace \alpha_j\right\rbrace$, aby $\left\lbrace z_j\right\rbrace$: 

$v=\sum_{i=1}^{n}\alpha_jz_j$ najlepiej przybliżał analityczne rozwiązanie.

Należy zwrócić uwagę na to, że podane metody nie mogą rozwiązać dowolnego równania różniczkowego czątkowego - to równanie musi się przedstawić w postaci dywergencyjnej (wykład 5).
Przykłady w tym notebooku będą oczywiście spełniały podane założenia.

## Przykład - Metoda Ritza / Reyleigha-Ritza
Mamy dane równanie różniczkowe:

$-u''(x)=-2$ dla $x\in (0,1)$

Warunki brzegowe: 
$u(0)=u(1)=0$

Wykorzystamy następujące funkcje bazowe: $z_1(x)=x(1-x)$ oraz $z_2(x)=x^2(1-x).$

Mamy równanie postaci $Au=f$. Aby uzyskać odpowiednie współczynniki $\alpha$ musimy zminimalizować funkcjonał:
$(Av,v)-2(f,v)$, co odpowiada funkcjonałowi:

$F(v)=-\int_0^1v''(x)v(x)dx +\int_0^14v(x)dx = \int_0^1\left( u'(x)\right)^2+4v(x)dx$

In [92]:
x, a1, a2 = symbols("x a1 a2")
z1 = x ** 2 * (1-x)
z2 = x ** 3 * (1-x)
v = a1 * z1 + a2 * z2
F = integrate(diff(v,x) ** 2 + 4 * v, (x, 0, 1))

r1 = [float(x) for x in Poly(diff(F, a1),(a1,a2)).coeffs()]
r2 = [float(x) for x in Poly(diff(F, a2),(a1,a2)).coeffs()]

# a1, a2, wyraz wolny
A = np.array([r1[:-1], r2[:-1]])
b = np.array([r1[-1], r2[-1]])
print(np.linalg.solve(A, b))

[ 3.         -2.33333333]


## Zadanie 2.
Rozwiąż równanie analitycznie. Sporządź wykres błędu (różnicy między rozwiązaniem dokładnym, a przybliżonym).

https://docs.sympy.org/latest/modules/plotting.html

## Zadanie 3.
Wypróbuj inny zestaw funkcji bazowych. Sporządź wykres zależności liczby funkcji bazowych w zależności od błędu 
aproksymacji dla liczby funkcji od 1 do 5. Niech funkcje będą postaci: $z_n=x^{n+1}(x-1)$.  

## Metoda Galerkina
Mamy dane równanie różniczkowe: $Au=f$.

Przemnóżmy skalarnie to równanie przez u: $(Au,u)=(f,u)$.

Wstawmy w miejce u nasze funkcje bazowe - stwórzmy macierz Grama G: 
- $(Az_1,z_1) = g_{11}$
- $(Az_1,z_2) = g_{12}$
- $(Az_m,z_n) = g_{mn}$
- ...

Aby uzyskać współczynniki $\alpha$ przy funkcjach bazowych naszego przyblożonego rozwiązania należy rozwiązać układ $G\alpha = b$.

$\alpha=[\alpha_1,\alpha_2,$...$]$

Natomiast wektor wyrazów wolnych będzie składał się z iloczynów: $b=[(f,z_1),(f,z_2),$...$]$

In [89]:
def my_norm(a, b):
    return integrate(diff(diff(a, x), x) * b, (x, 0, 1))
base_functions = [z1, z2]
G = np.array([[float(my_norm(x, y)) for x in base_functions] for y in base_functions])
b = np.array([float(integrate(-2 * z, (x, 0, 1))) for z in base_functions])
print(np.linalg.solve(G,b))

[ 3.         -2.33333333]


Jak można zauważyć otrzymujemy dokładnie takie samo rozwiązanie, jak to z wykorzystaniem metody Ritza.
## Zadanie 4
Wykorzystując metodę Galerkina rozwiąż równanie różniczkowe:

$\frac{d^2T}{dx^2} + 1000x^2 = 0$, $x\in[0,1]$

Przy następujących warunkach brzegowych: $T(0)=T(1)=0$

Analizowane równanie jest formą równania przewodnictwa cieplnego w izolowanym pręcie, z wewnętrznym źródłem energii. Wydajność tego źródła jest proporcjonalna do $x^2$.

## Zadanie 5
Wykorzystując metodę Ritza rozwiąż równanie różniczkowe:

$\Delta u=1$ w obszarze $D = [-1,1]\times[-1,1]$

$u|_{\partial D}=0$

Jaka jest postać funkcjonału, który będziemy minimalizować? Jaka jest postać rozwiązania przy wykorzystaniu jednej funkcji kształtu $z(x,y)=(1-x^2)(1-y^2)$?

